In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
import datetime as dt
import random

import folium
from folium import plugins
import ipywidgets
import geocoder
import geopy
from folium.plugins import MarkerCluster

from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=True)
from uszipcode import Zipcode

In [2]:
df = pd.read_csv("crime.csv")

In [4]:
df.dropna(subset = ["Lon"], inplace=True)
df.dropna(subset = ["Lat"], inplace=True)
df['Dispatch_Date'] = pd.to_datetime(df['Dispatch_Date'], errors='coerce')
include = df[(df.Dispatch_Date.dt.year == 2016) & (df.Dispatch_Date.dt.month == 5)]

In [5]:
include['day_of_week'] = include['Dispatch_Date'].dt.dayofweek


data_weekdays = include.loc[include['day_of_week'] <= 5 ]
data_weekends = include.loc[include['day_of_week'] > 5 ]


/Users/kasturivartak/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Dc_Dist,Psa,Dispatch_Date_Time,Dispatch_Date,Dispatch_Time,Hour,Dc_Key,Location_Block,UCR_General,Text_General_Code,Police_Districts,Month,Lon,Lat,day_of_week
138501,24,1,2016-05-16 10:27:00,2016-05-16,10:27:00,10,201624047512,2500 BLOCK CASTOR AVE,600.0,Theft from Vehicle,17.0,2016-05,-75.092576,39.992484,0
1197137,16,2,2016-05-16 18:21:00,2016-05-16,18:21:00,18,201616020537,100 BLOCK N 50TH ST,800.0,Other Assaults,12.0,2016-05,-75.220402,39.961738,0
1197431,16,1,2016-05-16 20:14:00,2016-05-16,20:14:00,20,201616020556,0 BLOCK N 39TH ST,800.0,Other Assaults,12.0,2016-05,-75.199497,39.957217,0
1221278,2,3,2016-05-16 12:43:00,2016-05-16,12:43:00,12,201602029133,2000 BLOCK VISTA ST,800.0,Other Assaults,2.0,2016-05,-75.062269,40.052819,0
1223745,25,1,2016-05-16 13:32:00,2016-05-16,13:32:00,13,201625040458,4200 BLOCK N 09TH ST,400.0,Aggravated Assault Firearm,18.0,2016-05,-75.140810,40.016979,0


In [6]:

location_weekdays=data_weekdays.groupby(['Lat','Lon','Text_General_Code']).size().reset_index(name='cnt').sort_values(by='cnt',ascending=False)
location_weekdays=location_weekdays.sort_values('cnt')

location_weekends=data_weekends.groupby(['Lat','Lon','Text_General_Code']).size().reset_index(name='cnt').sort_values(by='cnt',ascending=False)
location_weekends=location_weekends.sort_values('cnt')


In [7]:
def get_zipcode(lat, lon):
    result = search.by_coordinates(lat = lat, lng = lon, returns = 1)
    return result[0].zipcode

location_weekdays['zipcode'] = location_weekdays.apply(lambda x: get_zipcode(x.Lat,x.Lon), axis=1)
location_weekends['zipcode'] = location_weekends.apply(lambda x: get_zipcode(x.Lat,x.Lon), axis=1)


In [12]:
location_weekends['color']=location_weekdays['color']=location_weekdays['cnt'].apply(lambda count:"black" if count==53 else
                                        "grey" if count==42 else
                                        "red" if count==31 else
                                        "violet" if count==23 else
                                        "indianred" if count==21 else
                                        "red" if count==20 else
                                        "deeppink" if count==17 else
                                        "purple" if count==16 else
                                        "darkviolet" if count==15 else
                                        "slateblue" if count==14 else
                                        "green" if count==13 else
                                        "lightgreen" if count==12 else
                                        "aqua" if count==11 else
                                        "dodgerblue" if count==10 else
                                        "darkorange" if count==9 else
                                        "orange" if count==8 else
                                        "forestgreen" if count==7 else
                                        "limegreen" if count==6 else
                                        "rawgreen" if count==5 else
                                        "darkcyan" if count==4 else
                                        "blue" if count==3 else
                                        "red" if count==2 else
                                         "green")

location_weekends['size']=location_weekdays['size']=location_weekdays['cnt'].apply(lambda count:22 if count==53 else
                                        21 if count==42 else
                                        19 if count==31 else
                                        17 if count==23 else
                                        16 if count==21 else
                                        16 if count==20 else
                                        14 if count==17 else
                                        13 if count==16 else
                                        12 if count==15 else
                                        12 if count==14 else
                                        10 if count==13 else
                                        10 if count==12 else
                                        10 if count==11 else
                                        9 if count==10 else
                                        9 if count==9 else
                                        9 if count==8 else
                                        8 if count==7 else
                                        6 if count==6 else
                                        5 if count==5 else
                                        4 if count==4 else
                                        3 if count==3 else
                                        3 if count==2 else
                                        0.7)

In [14]:
m = folium.plugins.DualMap(location=(39.9527237,-75.1635262), tiles=None, zoom_start=11)
folium.TileLayer('openstreetmap').add_to(m.m1)
folium.TileLayer('openstreetmap').add_to(m.m2)
for lat,lon,code,zipcode,color,count,size in zip(location_weekdays['Lat'],location_weekdays['Lon'],location_weekdays['Text_General_Code'],location_weekdays['zipcode'],location_weekdays['color'],location_weekdays['cnt'],location_weekdays['size']):
            my_string = 'Type: {}, Zip: {}'.format(code, zipcode)
            folium.CircleMarker([lat, lon],
                            popup = folium.Popup((
    "Type: {type}<br>"
    "Zip: {zip}<br>"
   ).format(type=code,
            zip=zipcode),max_width=265),
                            radius=size,
                            color='b',
                            fill=True,
                            fill_opacity=0.7,
                            fill_color=color,
                           ).add_to(m.m1)
        
for lat,lon,code,zipcode,color,count,size in zip(location_weekends['Lat'],location_weekends['Lon'],location_weekends['Text_General_Code'],location_weekends['zipcode'],location_weekends['color'],location_weekends['cnt'],location_weekends['size']):
            my_string = 'Type: {}, Zip: {}'.format(code, zipcode)
            folium.CircleMarker([lat, lon],
                            popup = folium.Popup((
    "Type: {type}<br>"
    "Zip: {zip}<br>"
   ).format(type=code,
            zip=zipcode),max_width=265),
                            radius=size,
                            color='b',
                            fill=True,
                            fill_opacity=0.7,
                            fill_color=color,
                           ).add_to(m.m2)
    
m.save('PHL_compare_map.html')